In [1]:
!pip install hqq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 890.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia

In [3]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [7]:
# Step 1: Prepare Mixtral model

# Install and import libraries
import sys
import os

# Assuming 'src' is in the same directory as your current working directory in Colab
# Adjust the path accordingly if 'src' is located elsewhere
sys.path.append(os.path.abspath('./mixtral-offloading'))

# Fix numpy in colab
import numpy
from IPython.display import clear_output

# Fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

# Clone the repository and install requirements
!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

# Step 1: Prepare Mixtral model
# Install and import libraries
import numpy
from IPython.display import clear_output
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging
from transformers import TextStreamer

# Fix numpy in colab
import numpy
# Add the path where 'src' is located
sys.path.append(os.path.abspath('./mixtral-offloading'))
from src.build_model import OffloadConfig, QuantConfig, build_model

clear_output()


Login to huggingface

In [11]:
from huggingface_hub import notebook_login
notebook_login()

initialize model

In [13]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

RuntimeError: Error(s) in loading state_dict for MixtralForCausalLM:
	Missing key(s) in state_dict: "model.layers.0.self_attn.q_proj.meta", "model.layers.0.self_attn.k_proj.meta", "model.layers.0.self_attn.v_proj.meta", "model.layers.0.self_attn.o_proj.meta", "model.layers.1.self_attn.q_proj.meta", "model.layers.1.self_attn.k_proj.meta", "model.layers.1.self_attn.v_proj.meta", "model.layers.1.self_attn.o_proj.meta", "model.layers.2.self_attn.q_proj.meta", "model.layers.2.self_attn.k_proj.meta", "model.layers.2.self_attn.v_proj.meta", "model.layers.2.self_attn.o_proj.meta", "model.layers.3.self_attn.q_proj.meta", "model.layers.3.self_attn.k_proj.meta", "model.layers.3.self_attn.v_proj.meta", "model.layers.3.self_attn.o_proj.meta", "model.layers.4.self_attn.q_proj.meta", "model.layers.4.self_attn.k_proj.meta", "model.layers.4.self_attn.v_proj.meta", "model.layers.4.self_attn.o_proj.meta", "model.layers.5.self_attn.q_proj.meta", "model.layers.5.self_attn.k_proj.meta", "model.layers.5.self